# Sentiment Analysis Using LSTM

In [82]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

In [83]:
df=pd.read_csv('Ola_Dataset1.csv')

In [84]:
df.head()

,Unnamed: 0,tweet,Subjectivity,Polarity,Analysis,label
0,0,not that it will matter to you no zamato swi...,1.00,1.0,Positive,1.0
1,1,Happy church day ⛪⛪ Marriott Hotel Downtown A...,1.00,0.8,Positive,1.0
2,2,Over drivers in London register on Ola Great ...,0.75,0.8,Positive,1.0
3,3,It would be great if you chain me in this ap...,0.75,0.8,Positive,1.0
4,4,Just wait Sir will sooner you face a welcome ...,0.90,0.8,Positive,1.0


In [85]:
###Drop Nan Values
df=df.dropna()


In [86]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [87]:
## Get the Dependent features
y=df['label']

In [88]:
X.shape

(459, 5)

In [89]:
y.shape

(459,)

In [90]:
import tensorflow as tf

In [91]:
tf.__version__

'2.3.0'

In [92]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [93]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [94]:
messages=X.copy()

In [95]:
messages['tweet'][1]

'Happy church day ⛪⛪  Marriott Hotel Downtown Abu Dhabi  httpstcoOVDosxpg'

In [96]:
messages.reset_index(inplace=True)

In [97]:
import nltk
import re
from nltk.corpus import stopwords

In [98]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [99]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['tweet'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [100]:
corpus

['matter zamato swiggi ola amazon flipkart agra cut world mind u peak tourist season wonder messag tourist take home govt',
 'happi church day marriott hotel downtown abu dhabi httpstcoovdosxpg',
 'driver london regist ola great job httpstcocgjquutv',
 'would great chain appreci',
 'wait sir sooner face welcom fire shout hu',
 'thank pleasant httpstcoorjvhlu',
 'ola rental good',
 'surg also blame n cab aggreg food deliveri also good go httpstcocfuewdipg',
 'good advic',
 'honest never ever could success book morn hour heard day well time unistal ola',
 'lol okay',
 'kindli look paid googl pay yet paid cash httpstcojrnzbpi',
 'want regist case ola refund money kindli let process',
 'team dm queri kindli check thank',
 'amount debit account ride cash also paid driver ride kindli help',
 'dear fine cancel charg rs fault mine cancel made driver phone switch couldnt contact kindli look matter book id httpstcovrormznax',
 'dont want autom revert kindli call speak resolv issu',
 'sir alway p

In [101]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2315,
  3209,
  951,
  3687,
  4599,
  2458,
  1901,
  4631,
  226,
  2656,
  4390,
  4169,
  595,
  3653,
  81,
  4514,
  595,
  1136,
  2231,
  1566],
 [60, 3107, 4595, 1146, 3564, 3423, 3310, 3480, 2400],
 [3967, 4892, 944, 3687, 4722, 3824, 4435],
 [2123, 4722, 4458, 3061],
 [1191, 3865, 2186, 4871, 2807, 4529, 911, 2672],
 [4340, 2154, 1945],
 [3687, 3834, 2964],
 [3722, 3769, 1000, 2031, 4584, 4013, 2905, 4933, 3769, 2964, 877, 2625],
 [2964, 1689],
 [633, 383, 2059, 814, 4771, 4976, 90, 2972, 3348, 4595, 38, 495, 4627, 3687],
 [680, 1688],
 [4436, 1953, 4244, 30, 3569, 2122, 4244, 4974, 3384],
 [1813, 944, 3504, 3687, 1392, 4998, 4436, 4872, 4615],
 [1981, 4030, 174, 4436, 2315, 4340],
 [4937, 3133, 2083, 4138, 4974, 3769, 4244, 3967, 4138, 4436, 444],
 [2998,
  4995,
  758,
  1382,
  4224,
  3440,
  760,
  758,
  2572,
  3967,
  2386,
  1600,
  2629,
  4982,
  4436,
  1953,
  2315,
  4976,
  1644,
  2070],
 [4856, 1813, 3446, 2801, 4436, 3041, 2416, 1021, 3776],
 [3865,
  477

### Embedding Representation

In [102]:
sent_length=40
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1136 2231 1566]
 [   0    0    0 ... 3310 3480 2400]
 [   0    0    0 ... 4722 3824 4435]
 ...
 [   0    0    0 ...  389 4308 4425]
 [   0    0    0 ...  401 2462 3661]
 [   0    0    0 ... 2566 3687 2985]]


In [103]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0, 2315, 3209,
        951, 3687, 4599, 2458, 1901, 4631,  226, 2656, 4390, 4169,  595,
       3653,   81, 4514,  595, 1136, 2231, 1566])

In [104]:
## Creating model
embedding_vector_features=50
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 40, 50)            250000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________
None


In [105]:
len(embedded_docs),y.shape

(459, (459,))

In [106]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [107]:
X_final.shape,y_final.shape

((459, 40), (459,))

In [115]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.4, random_state=101)

### Model Training

In [116]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=30,batch_size=50)

Epoch 1/30
6/6 [==============================] - 0s 20ms/step - loss: 0.5334 - accuracy: 0.8255 - val_loss: 0.3606 - val_accuracy: 0.8587
Epoch 2/30
6/6 [==============================] - 0s 19ms/step - loss: 0.3704 - accuracy: 0.8764 - val_loss: 0.4158 - val_accuracy: 0.8478
Epoch 3/30
6/6 [==============================] - 0s 17ms/step - loss: 0.2877 - accuracy: 0.9055 - val_loss: 0.3632 - val_accuracy: 0.8913
Epoch 4/30
6/6 [==============================] - 0s 19ms/step - loss: 0.1333 - accuracy: 0.9600 - val_loss: 0.3699 - val_accuracy: 0.8641
Epoch 5/30
6/6 [==============================] - 0s 19ms/step - loss: 0.1170 - accuracy: 0.9782 - val_loss: 0.3671 - val_accuracy: 0.8750
Epoch 6/30
6/6 [==============================] - 0s 19ms/step - loss: 0.0960 - accuracy: 0.9818 - val_loss: 0.3659 - val_accuracy: 0.8804
Epoch 7/30
6/6 [==============================] - 0s 18ms/step - loss: 0.0744 - accuracy: 0.9891 - val_loss: 0.3718 - val_accuracy: 0.8804
Epoch 8/30
6/6 [===========

### Performance Metrics And Accuracy

In [117]:
y_pred=model.predict_classes(X_test)

In [118]:
from sklearn.metrics import confusion_matrix

In [119]:
confusion_matrix(y_test,y_pred)

array([[ 56,  18],
       [  0, 110]], dtype=int64)

In [120]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9021739130434783

In [121]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       1.00      0.76      0.86        74
         1.0       0.86      1.00      0.92       110

    accuracy                           0.90       184
   macro avg       0.93      0.88      0.89       184
weighted avg       0.92      0.90      0.90       184

